In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# eps for making value a bit greater than 0 later on
eps = np.finfo(float).eps

In [92]:
(9/14)*np.log2(9/14)+5/14*np.log2(5/14)

-0.9402859586706311

In [2]:
data = pd.read_csv('PlayTennis.csv')

In [3]:
data

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [53]:
def entropy(df):
    Class = df.keys()[-1] 
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [55]:
entropy(data)

0.9402859586706311

In [56]:
def entropy_attribute(df,attribute):
    Class = df.keys()[-1]   
    target_variables = df[Class].unique()  
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
                num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
                den = len(df[attribute][df[attribute]==variable])
                fraction = num/(den+eps)
                entropy += -fraction*np.log2(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

In [57]:
entropy_attribute(data,'Outlook')

0.6935361388961914

In [58]:
def information_gain(df):
    IG = []
    for key in df.keys()[:-1]:
        IG.append(entropy(df)-entropy_attribute(df,key))
    return IG, df.keys()[np.argmax(IG)]

In [62]:
information_gain(data)

([0.24674981977443977,
  0.029222565658955535,
  0.15183550136234225,
  0.048127030408270155],
 'Outlook')

### The Root is Outlook

In [85]:
def get_subtable(df, node, value):
    df2 = df[df[node] == value].reset_index(drop=True)
    df2.drop(node,axis=1,inplace=True)
    return df2

In [78]:
dt2_1 = get_subtable(data,'Outlook','Sunny')
dt2_1

,Temperature,Humidity,Wind,Play Tennis
0,Hot,High,Weak,No
1,Hot,High,Strong,No
2,Mild,High,Weak,No
3,Cool,Normal,Weak,Yes
4,Mild,Normal,Strong,Yes


In [86]:
information_gain(dt2_1)

([0.5709505944546689, 0.9709505944546682, 0.019973094021975557], 'Humidity')

### The Final Leaf is Humidity for Outlook is Sunny

In [79]:
dt2_2 = get_subtable(data,'Outlook','Overcast')
dt2_2

,Temperature,Humidity,Wind,Play Tennis
0,Hot,High,Weak,Yes
1,Cool,Normal,Strong,Yes
2,Mild,High,Strong,Yes
3,Hot,Normal,Weak,Yes


### The Final Leaf is Outlook is Overcast

In [80]:
dt2_3 = get_subtable(data,'Outlook','Rain')
dt2_3

,Temperature,Humidity,Wind,Play Tennis
0,Mild,High,Weak,Yes
1,Cool,Normal,Weak,Yes
2,Cool,Normal,Strong,No
3,Mild,Normal,Weak,Yes
4,Mild,High,Strong,No


In [89]:
information_gain(dt2_3)

([0.019973094021975557, 0.019973094021975557, 0.9709505944546682], 'Wind')

In [90]:
get_subtable(dt2_3,'Wind','Weak')

,Temperature,Humidity,Play Tennis
0,Mild,High,Yes
1,Cool,Normal,Yes
2,Mild,Normal,Yes


### The Final Leaf is Wind is Weak

In [91]:
get_subtable(dt2_3,'Wind','Strong')

,Temperature,Humidity,Play Tennis
0,Cool,Normal,No
1,Mild,High,No


### THe Final Leaf is Wind is Strong